### *cis*- and *trans*-QTL mapping with tensorQTL

This notebook provides examples for running *cis*- and *trans*-QTL mapping with tensorQTL, using open-access data from the [GEUVADIS](https://www.ebi.ac.uk/arrayexpress/experiments/E-GEUV-1/) project.

#### Requirements
An environment configured with a GPU and ~50GB of memory.

#### Test dataset

*Note: these files are provided for testing/benchmarking purposes only. They do not constitute an official release from the GEUVADIS project, and no quality-control was applied.*

Genotypes in PLINK and VCF format, and normalized expression data are available [here](https://personal.broadinstitute.org/francois/geuvadis/).

Alternatively, to download the files required for these examples, uncomment and run the cell below.

In [1]:
# !wget https://personal.broadinstitute.org/francois/geuvadis/GEUVADIS.445_samples.GRCh38.20170504.maf01.filtered.bed
# !wget https://personal.broadinstitute.org/francois/geuvadis/GEUVADIS.445_samples.GRCh38.20170504.maf01.filtered.bim
# !wget https://personal.broadinstitute.org/francois/geuvadis/GEUVADIS.445_samples.GRCh38.20170504.maf01.filtered.fam   
# !wget https://personal.broadinstitute.org/francois/geuvadis/GEUVADIS.445_samples.covariates.txt
# !wget https://personal.broadinstitute.org/francois/geuvadis/GEUVADIS.445_samples.expression.bed.gz

In [2]:
import pandas as pd
import numpy as np
import tensorqtl
from tensorqtl import genotypeio, cis, trans
import matplotlib.pyplot as plt

# define paths to data
plink_prefix_path = 'GEUVADIS.445_samples.GRCh38.20170504.maf01.filtered'
expression_bed = 'GEUVADIS.445_samples.expression.bed.gz'
covariates_file = 'GEUVADIS.445_samples.covariates.txt'
prefix = 'GEUVADIS.445_samples'

# load phenotypes and covariates
phenotype_df, phenotype_pos_df = tensorqtl.read_phenotype_bed(expression_bed)
covariates_df = pd.read_csv(covariates_file, sep='\t', index_col=0).T

# PLINK reader for genotypes
pr = genotypeio.PlinkReader(plink_prefix_path)
genotype_df = pd.DataFrame(pr.get_all_genotypes(), index=pr.bim['snp'], columns=pr.fam['iid'])
variant_df = pr.bim.set_index('snp')[['chrom', 'pos']]

Mapping files: 100%|██████████| 3/3 [00:21<00:00,  9.54s/it]


### *cis*-QTL: nominal p-values for all variant-phenotype pairs

In [3]:
# map all cis-associations (results for each chromosome are written to file)

# all genes
# cis.map_nominal(genotype_df, variant_df, phenotype_df, phenotype_pos_df, covariates_df, prefix)

# genes on chr18
cis.map_nominal(genotype_df, variant_df, phenotype_df.loc[phenotype_pos_df['chr']=='chr18'],
                phenotype_pos_df.loc[phenotype_pos_df['chr']=='chr18'], covariates_df, prefix)

cis-QTL mapping: nominal associations for all variant-phenotype pairs
  * 445 samples
  * 301 phenotypes
  * 26 covariates
  * 13380684 variants
  * checking phenotypes: 301/301
  * Computing associations
    Mapping chromosome chr18
    processing phenotype 301/301
    time elapsed: 0.05 min
  * writing output
done.


In [4]:
# load results
pairs_df = pd.read_parquet('{}.cis_qtl_pairs.chr18.parquet'.format(prefix))
pairs_df.head()

,phenotype_id,variant_id,tss_distance,maf,ma_samples,ma_count,pval_nominal,slope,slope_se
0,ENSG00000263006.6,chr18_10644_C_G_b38,-98421,0.016854,15,15,0.577392,-0.118906,0.213232
1,ENSG00000263006.6,chr18_10847_C_A_b38,-98218,0.019101,17,17,0.142049,-0.299512,0.203613
2,ENSG00000263006.6,chr18_11275_G_A_b38,-97790,0.024719,22,22,0.734261,0.057100,0.168095
3,ENSG00000263006.6,chr18_11358_G_A_b38,-97707,0.024719,22,22,0.734261,0.057100,0.168095
4,ENSG00000263006.6,chr18_11445_G_A_b38,-97620,0.023596,21,21,0.592376,0.092131,0.171947


### *cis*-QTL: empirical p-values for phenotypes

In [5]:
# all genes
# cis_df = cis.map_cis(genotype_df, variant_df, phenotype_df, phenotype_pos_df, covariates_df)

# genes on chr18
cis_df = cis.map_cis(genotype_df, variant_df, phenotype_df.loc[phenotype_pos_df['chr']=='chr18'],
                     phenotype_pos_df.loc[phenotype_pos_df['chr']=='chr18'], covariates_df)

cis-QTL mapping: empirical p-values for phenotypes
  * 445 samples
  * 301 phenotypes
  * 26 covariates
  * 13380684 variants
  * checking phenotypes: 301/301
  * computing permutations
    processing phenotype 301/301
  Time elapsed: 0.51 min
done.


In [6]:
cis_df.head()

,num_var,beta_shape1,beta_shape2,true_df,pval_true_df,variant_id,tss_distance,ma_samples,ma_count,maf,ref_factor,pval_nominal,slope,slope_se,pval_perm,pval_beta
phenotype_id,,,,,,,,,,,,,,,,
ENSG00000263006.6,6126,1.038505,1123.368896,374.302887,7.786380e-40,chr18_112535_G_A_b38,3470,212,251,0.282022,1,3.473041e-44,0.726806,0.046150,0.000100,3.517118e-38
ENSG00000101557.14,6361,1.067356,1061.277466,366.354645,7.109856e-11,chr18_210698_T_C_b38,52315,192,222,0.249438,1,3.529043e-12,-0.191590,0.026736,0.000100,2.426521e-08
ENSG00000079134.11,6929,1.058589,1190.084473,371.821655,3.836204e-08,chr18_243547_T_A_b38,-24503,293,383,0.430337,1,5.776206e-09,-0.122455,0.020590,0.000100,2.477086e-05
ENSG00000263884.1,6929,1.045611,1324.890747,378.409332,6.550194e-04,chr18_584440_G_C_b38,316292,81,88,0.098876,1,3.468841e-04,-0.331377,0.091862,0.562044,5.594898e-01
ENSG00000158270.11,8143,1.040845,1431.478149,378.279694,1.854220e-09,chr18_519222_C_T_b38,18500,108,115,0.129213,1,2.780035e-10,-0.386568,0.059763,0.000100,1.543362e-06


### *trans*-QTL mapping

In [7]:
# run mapping
# to limit output size, only associations with p-value <= 1e-5 are returned
trans_df = trans.map_trans(genotype_df, phenotype_df, covariates_df, batch_size=20000,
                           return_sparse=True, pval_threshold=1e-5, maf_threshold=0.05)

trans-QTL mapping
  * 445 samples
  * 19836 phenotypes
  * 26 covariates
  * 13380684 variants
    processing batch 670/670
    elapsed time: 1.11 min
  * 7627084 variants passed MAF >= 0.05 filtering
done.


In [8]:
# remove cis-associations
trans_df = trans.filter_cis(trans_df, phenotype_pos_df.T.to_dict(), variant_df, window=5000000)

In [9]:
trans_df.head()

,variant_id,phenotype_id,pval,maf
0,chr1_10177_A_AC_b38,ENSG00000169203.16,6.134262e-06,0.428090
2,chr1_30923_G_T_b38,ENSG00000278668.1,8.346116e-06,0.123595
3,chr1_30923_G_T_b38,ENSG00000156531.16,1.818096e-06,0.123595
4,chr1_47159_T_C_b38,ENSG00000185324.21,7.856671e-25,0.056180
5,chr1_49554_A_G_b38,ENSG00000271155.1,3.945799e-06,0.051685
